In [1]:
!sudo yum update -y 
!curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.rpm.sh | sudo bash
!sudo yum install git-lfs git -y

Loaded plugins: dkms-build-requires, extras_suggestions, langpacks, priorities,
              : update-motd, versionlock
amzn2-core                                               | 3.7 kB     00:00     
amzn2extra-docker                                        | 3.0 kB     00:00     
amzn2extra-kernel-5.10                                   | 3.0 kB     00:00     
centos-extras                                            | 2.9 kB     00:00     
copr:copr.fedorainfracloud.org:vbatts:shadow-utils-newxi | 3.3 kB     00:00     
https://download.docker.com/linux/centos/2/x86_64/stable/repodata/repomd.xml: [Errno 14] HTTPS Error 404 - Not Found
Trying other mirror.
libnvidia-container/x86_64/signature                     |  833 B     00:00     
libnvidia-container/x86_64/signature                     | 2.1 kB     00:00 !!! 
neuron                                                   | 2.9 kB     00:00     
nvidia-container-runtime/x86_64/signature                |  833 B     00:00     
nvidia-conta

In [2]:
import sagemaker
import boto3
sess = sagemaker.Session()
# sagemaker session bucket -> used for uploading data, models and logs
# sagemaker will automatically create this bucket if it not exists
sagemaker_session_bucket=None
if sagemaker_session_bucket is None and sess is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = sess.default_bucket()

try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")

sagemaker role arn: arn:aws:iam::905847418383:role/service-role/AmazonSageMaker-ExecutionRole-20220310T133301
sagemaker bucket: sagemaker-us-east-1-905847418383
sagemaker session region: us-east-1


In [19]:
repository = "bigscience/bloom-3b"
model_id=repository.split("/")[-1]
s3_location=f"s3://{sess.default_bucket()}/{model_id}/model.tar.gz"

In [20]:
!git lfs install
!git clone https://huggingface.co/$repository

Updated Git hooks.
Git LFS initialized.
Cloning into 'bloom-3b'...
remote: Enumerating objects: 83, done.
remote: Counting objects: 100% (83/83), done.
remote: Compressing objects: 100% (82/82), done.
remote: Total 83 (delta 44), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (83/83), 28.84 KiB | 1.37 MiB/s, done.


In [21]:
!mkdir code

In [22]:
%%writefile code/inference.py
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch


def model_fn(model_dir):
    model_8bit = AutoModelForCausalLM.from_pretrained(model_dir, device_map="auto", load_in_8bit=True)
    tokenizer = AutoTokenizer.from_pretrained(model_dir)
    return model_8bit, tokenizer


def predict_fn(data, model_and_tokenizer):
    model, tokenizer = model_and_tokenizer
    text = data.pop("inputs", data)
    encoded_input = tokenizer(text, return_tensors='pt')
    output_sequences = model.generate(input_ids=encoded_input['input_ids'].cuda(), **data)
    return tokenizer.decode(output_sequences[0], skip_special_tokens=True)

Writing code/inference.py


In [23]:
%%writefile code/requirements.txt
bitsandbytes
accelerate
git+https://github.com/huggingface/transformers.git@main#egg=transformers

Writing code/requirements.txt


In [24]:
!cp -r code/ $model_id/code/

In [25]:
%cd $model_id
!tar zcvf model.tar.gz *

/home/ec2-user/SageMaker/sm-bnb-transformers/bloom-3b
code/
code/requirements.txt
code/inference.py
config.json
LICENSE
pytorch_model.bin
README.md
special_tokens_map.json
tokenizer_config.json
tokenizer.json


In [26]:
!aws s3 cp model.tar.gz $s3_location

upload: ./model.tar.gz to s3://sagemaker-us-east-1-905847418383/bloom-3b/model.tar.gz


In [27]:
from sagemaker.utils import name_from_base
from sagemaker.huggingface.model import HuggingFaceModel

endpoint_name = name_from_base(f"{model_id}-bnb")

# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
    model_data=s3_location,
    role=role,
    transformers_version="4.17",
    pytorch_version="1.10",
    py_version='py38',
)

# deploy the endpoint endpoint
predictor = huggingface_model.deploy(
    initial_instance_count=1,
    instance_type="ml.g4dn.xlarge",
    endpoint_name=endpoint_name
)

-------------!

In [47]:
data = {
    "inputs": "Transformers with bnb-Int8 works best on",
    "do_sample": True,
    "temperature": 0.5,
}

res = predictor.predict(data=data)
print(res)

Transformers with bnb-Int8 works best on my android phone. I have tried to use
